In [1]:
# Here you go Stephanie!

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.patheffects as pe

In [3]:
# --------------------------
# Style helpers (colorblind-safe)
# --------------------------
BLUE = "#1f77b4"      # renewables
GRAY = "#4D4D4D"      # CO2 proxy (non-renewable)
GRAY_FILL = "#d9d9d9"

def fmt_pct(x, pos): return f"{x:.0f}%"
def fmt_int(x, pos): return f"{x:.0f}"

# --------------------------
# Load & shape (dataset-only)
# --------------------------
def load_country_year_totals(csv_path: Path) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # Only Electricity Generation (keeps GWh consistent)
    df = df[df["Indicator"].str.contains("Electricity Generation", case=False, na=False)]
    # Keep totals only
    df = df[df["Energy_Type"].isin(["Total Renewable", "Total Non-Renewable"])]

    # Year columns like F2000..F2024
    year_cols = [c for c in df.columns if isinstance(c, str) and c.startswith("F") and c[1:].isdigit()]
    long = df.melt(
        id_vars=["Country", "Energy_Type"],
        value_vars=year_cols,
        var_name="FYear",
        value_name="Value"
    ).dropna(subset=["Value"])

    long["Year"] = long["FYear"].str[1:].astype(int)
    long = long[(long["Year"] >= 2000) & (long["Year"] <= 2024)]

    pvt = (
        long
        .pivot_table(index=["Country","Year"], columns="Energy_Type", values="Value", aggfunc="sum")
        .reset_index()
        .rename(columns={"Total Renewable":"gen_renew_gwh", "Total Non-Renewable":"gen_nonrenew_gwh"})
    )

    pvt["gen_total_gwh"] = pvt[["gen_renew_gwh","gen_nonrenew_gwh"]].sum(axis=1)
    pvt["renew_share"] = pvt["gen_renew_gwh"] / pvt["gen_total_gwh"]
    return pvt

# --------------------------
# Visual 1 — Global (or regional) line chart
# --------------------------
def global_lines(df: pd.DataFrame) -> pd.DataFrame:
    g = (
        df.groupby("Year")
          .agg(gen_renew_gwh=("gen_renew_gwh","sum"),
               gen_nonrenew_gwh=("gen_nonrenew_gwh","sum"),
               gen_total_gwh=("gen_total_gwh","sum"))
          .reset_index()
    )
    g["renew_share_global"] = g["gen_renew_gwh"] / g["gen_total_gwh"]

    # Non-renewable index (2000 = 100). If first year missing, use earliest available.
    base = g.loc[g["Year"].idxmin(), "gen_nonrenew_gwh"]
    g["nonrenew_index_2000"] = g["gen_nonrenew_gwh"] / base * 100.0
    return g

def plot_visual_for_line_global(g: pd.DataFrame, outpath: Path):
    df = g.copy()

    fig, ax1 = plt.subplots(figsize=(10.4, 6))
    fig.patch.set_facecolor("white")

    # --- Left axis: Renewable share (%)
    ax1.plot(df["Year"], df["renew_share_global"]*100, color=BLUE, lw=3, solid_capstyle="round")
    ax1.set_ylabel("Renewable share of electricity (%)", color=BLUE)
    ax1.tick_params(axis="y", colors=BLUE)
    ax1.yaxis.set_major_formatter(FuncFormatter(fmt_pct))
    ax1.set_ylim(0, max(100, (df["renew_share_global"].max()*100)*1.05))  # zero baseline

    # --- Right axis: Non-renewable index (2000=100) as CO₂ proxy
    ax2 = ax1.twinx()
    # light fill under gray line (pre-attentive area cue without stealing attention)
    ax2.fill_between(df["Year"], 0, df["nonrenew_index_2000"], color=GRAY_FILL, alpha=1.0, zorder=0)
    ax2.plot(df["Year"], df["nonrenew_index_2000"], color=GRAY, lw=2.5, solid_capstyle="round")
    ax2.set_ylabel("Non-renewable electricity (index, 2000 = 100)", color=GRAY)
    ax2.tick_params(axis="y", colors=GRAY)
    ax2.yaxis.set_major_formatter(FuncFormatter(fmt_int))
    ax2.set_ylim(0, df["nonrenew_index_2000"].max()*1.10)  # zero baseline

    # --- Direct labels near the START of each series
    halo = [pe.withStroke(linewidth=3.5, foreground="white")]
    x0 = df["Year"].iloc[0]
    ax1.text(x0, df["renew_share_global"].iloc[0]*100 + 1.5, "Renewable share", color=BLUE,
             fontsize=12, weight="bold", ha="left", va="bottom", path_effects=halo)
    ax2.text(x0, df["nonrenew_index_2000"].iloc[0] + 3, "Non-renewable (CO₂ proxy)", color=GRAY,
             fontsize=12, weight="bold", ha="left", va="bottom", path_effects=halo)

    # subtle grid
    ax1.grid(True, axis="y", alpha=0.20)

    ax1.set_title("Globally, Renewable Share Rises While Non-Renewable Power Slows/Declines (2000–2024)")
    ax1.set_xlabel("Year")
    fig.tight_layout()
    fig.savefig(outpath, dpi=320, bbox_inches="tight")
    plt.close(fig)

# --------------------------
# VISUAL 2 — Diverging bar chart
# --------------------------
def plot_visual_for_bar(df: pd.DataFrame, outpath: Path, top_k=12):
    # --- compute changes ---
    def _changes(g):
        g = g.sort_values("Year")
        a, b = g.iloc[0], g.iloc[-1]
        d_renew = (b["renew_share"] - a["renew_share"]) * 100.0
        red_nonrenew = (a["gen_nonrenew_gwh"] - b["gen_nonrenew_gwh"]) / max(a["gen_nonrenew_gwh"], 1e-9) * 100.0
        return pd.Series({"d_renew_pp": d_renew, "reduction_nonrenew_pct": red_nonrenew})

    ch = df.groupby("Country", as_index=False).apply(_changes).dropna()
    ch.replace([np.inf, -np.inf], np.nan, inplace=True)
    ch = ch.dropna(subset=["d_renew_pp","reduction_nonrenew_pct"])

    top = ch.sort_values("d_renew_pp", ascending=False).head(top_k).reset_index(drop=True)
    top = top.iloc[::-1]  # biggest at top visually
    y_pos = np.arange(len(top))

    # --- plot (safe sizing) ---
    fig, ax = plt.subplots(figsize=(10, 7), constrained_layout=True)
    fig.patch.set_facecolor("white")

    b1 = ax.barh(y_pos - 0.2, top["d_renew_pp"], height=0.38, color=BLUE, alpha=0.95)
    b2 = ax.barh(y_pos + 0.2, top["reduction_nonrenew_pct"], height=0.38, color=GRAY, alpha=0.85)

    # numeric labels INSIDE or just past the bar end, but clipped to axes
    ax.bar_label(b1, labels=[f"{v:.1f} pp" for v in top["d_renew_pp"]], padding=3, color=BLUE, fontsize=8, clip_on=True)
    ax.bar_label(b2, labels=[f"{v:.1f} %"  for v in top["reduction_nonrenew_pct"]], padding=3, color=GRAY, fontsize=8, clip_on=True)

    # axes & grid
    ax.set_yticks(y_pos)
    ax.set_yticklabels(top["Country"], fontsize=9)
    xmax = float(max(top["d_renew_pp"].max(), top["reduction_nonrenew_pct"].max()))
    ax.set_xlim(0, xmax * 1.1)        # tight x-limits so labels stay inside
    ax.margins(x=0.02)                # tiny margin
    ax.grid(True, axis="x", alpha=0.2)
    ax.tick_params(axis="y", length=0)
    ax.set_xlabel("Change from 2000 to 2024")
    ax.set_title("Stronger Renewable Growth Tracks Larger Cuts in Non-Renewable Generation (2000→2024)")

    # NOTE: do NOT use bbox_inches='tight' here
    fig.savefig(outpath, dpi=200)
    plt.close(fig)


# ---- In your main() ----
if __name__ == "__main__":
    DATA_DIR = Path("../data")
    PLOTS_DIR = Path("../plots")
    PLOTS_DIR.mkdir(parents=True, exist_ok=True)

    SRC = DATA_DIR / "Renewable_Energy.csv"
    df = load_country_year_totals(SRC)

    # Visual 1 (line)
    g = global_lines(df)
    plot_visual_for_line_global(g, PLOTS_DIR / "visual1_for_line_global.png")

    # Visual 2 (bar)
    out2 = PLOTS_DIR / "visual2_for_bar.png"
    plot_visual_for_bar(df, out2, top_k=12)
    print(f"Saved: {out2}")



Saved: ../plots/visual2_for_bar.png


In [4]:
# --- color palette ---
BLUE = "#1f77b4"   # renewables
GRAY = "#4D4D4D"   # fossil
LIGHT_GRAY = "#d9d9d9"

# ----------------------
# Load + transform IMF Renewable data
# ----------------------
def load_country_year_totals(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    df = df[df["Indicator"].str.contains("Electricity Generation", case=False, na=False)]
    df = df[df["Energy_Type"].isin(["Total Renewable", "Total Non-Renewable"])]

    year_cols = [c for c in df.columns if c.startswith("F") and c[1:].isdigit()]
    long = df.melt(
        id_vars=["Country", "Energy_Type"],
        value_vars=year_cols,
        var_name="FYear",
        value_name="Value"
    ).dropna(subset=["Value"])

    long["Year"] = long["FYear"].str[1:].astype(int)
    long = long[(long["Year"] >= 2000) & (long["Year"] <= 2024)]

    pvt = (
        long.pivot_table(index=["Country", "Year"], columns="Energy_Type", values="Value", aggfunc="sum")
        .reset_index()
        .rename(columns={"Total Renewable": "gen_renew_gwh", "Total Non-Renewable": "gen_nonrenew_gwh"})
    )
    pvt["gen_total"] = pvt[["gen_renew_gwh", "gen_nonrenew_gwh"]].sum(axis=1)
    pvt["renew_share"] = pvt["gen_renew_gwh"] / pvt["gen_total"]
    pvt["nonrenew_share"] = 1 - pvt["renew_share"]
    return pvt

# ----------------------
# Visual 1: 100% composition (renewable vs non-renewable)
# ----------------------
def plot_visual1_stacked(g: pd.DataFrame, outpath: Path):
    gsum = (
        g.groupby("Year")[["gen_renew_gwh", "gen_nonrenew_gwh"]].sum().reset_index()
    )
    gsum["renew_share"] = gsum["gen_renew_gwh"] / (gsum["gen_renew_gwh"] + gsum["gen_nonrenew_gwh"])
    gsum["nonrenew_share"] = 1 - gsum["renew_share"]

    fig, ax = plt.subplots(figsize=(10, 6), constrained_layout=True)
    fig.patch.set_facecolor("white")

    # 100% composition stack
    ax.stackplot(
        gsum["Year"],
        gsum["renew_share"] * 100,
        gsum["nonrenew_share"] * 100,
        colors=[BLUE, LIGHT_GRAY],
        alpha=0.95
    )

    # axes
    ax.set_ylim(0, 100)
    first_year = int(gsum["Year"].min())
    last_year  = int(gsum["Year"].max())
    right_pad  = 0.8  # give a little room for endpoint labels
    ax.set_xlim(first_year, last_year + right_pad)
    ax.set_ylabel("Share of total electricity (%)")
    ax.set_xlabel("Year")
    ax.grid(True, axis="y", alpha=0.20)

    # --- endpoint labels with clash-avoidance ---
    halo = [pe.withStroke(linewidth=3.5, foreground="white")]

    y_renew_end = float(gsum["renew_share"].iloc[-1] * 100)
    y_fossil_end = float(gsum["nonrenew_share"].iloc[-1] * 100)

    # If labels are too close, nudge them apart by a few pct points
    if abs(y_renew_end - (100 - y_fossil_end)) < 8:
        # push renew a bit down and fossil a bit up
        renew_nudge = -3.5
        fossil_nudge = +3.5
    else:
        renew_nudge = 0.0
        fossil_nudge = 0.0

    # Place labels just outside the plotting area on the right
    label_x = last_year + 0.35

    ax.text(
        label_x, y_renew_end + renew_nudge,
        "Renewables",
        color=BLUE, fontsize=12, weight="bold",
        ha="left", va="center",
        path_effects=halo, clip_on=False
    )

    ax.text(
        label_x, 100 - y_renew_end + fossil_nudge,  # equals y_fossil_end
        "Fossil / Non-Renewable",
        color=GRAY, fontsize=12, weight="bold",
        ha="left", va="center",
        path_effects=halo, clip_on=False
    )

    # title
    ax.set_title("As Renewables Rise, Fossil Power Plateaus (2000–2024)", fontsize=13, pad=10)

    fig.savefig(outpath, dpi=320)
    plt.close(fig)

# ----------------------
# Main
# ----------------------
if __name__ == "__main__":
    DATA_DIR = Path("../data")
    PLOTS_DIR = Path("../plots")
    PLOTS_DIR.mkdir(parents=True, exist_ok=True)

    SRC = DATA_DIR / "Renewable_Energy.csv"
    df = load_country_year_totals(SRC)

    plot_visual1_stacked(df, PLOTS_DIR / "visual3_for_stacked.png")
    print("Saved stacked and slope visuals.")


Saved stacked and slope visuals.
